In [2]:
from uuid import uuid4
import pandas as pd
from langchain_core.documents import Document
import os
from model_config import VietnameseEmbeddings
from elasticsearch import Elasticsearch
# Kết nối tới Elasticsearch
es = Elasticsearch(
    "https://my-elasticsearch-project-fc9fd1.es.ap-southeast-1.aws.elastic.cloud:443",
    api_key="aXY4NkpKUUJaNVNfUEdnZHdVZ186MExmVk1iclZRWWlrS1hpeDRhOWRGUQ=="
)
index = "base"
embeddings = VietnameseEmbeddings()
excel = r"D:\DATN\QA_System\eval\testset\final_dataset_qa.xlsx"
chunks = pd.read_excel(excel)

c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Initializing Vietnamese embedding model: keepitreal/vietnamese-sbert


c:\Users\PC\anaconda3\envs\rag\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
import json
import pandas as pd
import time
from tqdm import tqdm
from elasticsearch import Elasticsearch

def search(query, top_k=1):
    # Add delay to avoid API overload (adjust as needed)
    time.sleep(0.1)  # 100ms delay between requests
    result = es.search(
		index=index,
		body={
			"query": {
					"match": {
							"text": query
					}
			},
      "size": top_k
		},
	)
        
    
    res = [hit['_source']["metadata"] for hit in result['hits']['hits']]
    return res

def evaluate(excel_path, top_k_values=[1, 5, 10, 20]):
    chunk = pd.read_excel(excel_path).head(10)
    total_questions = len(chunk)
    
    # Khởi tạo bộ đếm điểm
    hits = {k: 0 for k in top_k_values}
    mrr = {k: 0 for k in top_k_values}
    
    # Add progress bar
    progress_bar = tqdm(chunk.iterrows(), total=total_questions, desc="Evaluating questions")
    
    for _, row in progress_bar:
        question = row['question']
        metadata = row['metadata']
        if isinstance(metadata, str):
            metadata = metadata.replace("'", "\"")
            try:
                metadata = json.loads(metadata)
            except json.JSONDecodeError:
                continue
        correct_source = metadata
        
        # Update progress bar description with current question
        progress_bar.set_description(f"Processing: {question[:30]}...")
        
        # Thực hiện tìm kiếm
        search_results = search(question, top_k=max(top_k_values))
        
        # Kiểm tra kết quả và tính điểm
        sources = [result.get("source", "") for result in search_results]
        print("correct source:",correct_source)
        print("search_results:",search_results)
        print("sources:",sources)
        for k in top_k_values:
            if correct_source in sources[:k]:
                hits[k] += 1
                rank = sources.index(correct_source) + 1
                mrr[k] += 1 / rank if rank <= k else 0
        
        # Update progress bar postfix with current stats
        progress_bar.set_postfix({f'Hit@{max(top_k_values)}': f'{(hits[max(top_k_values)] / (progress_bar.n + 1)):.2f}'})
    
    # Close progress bar
    progress_bar.close()
    
    # Tính tỷ lệ điểm
    hit_ratios = {k: hits[k] / total_questions for k in top_k_values}
    mrr_scores = {k: mrr[k] / total_questions for k in top_k_values}
    
    # Ghi kết quả vào Excel với timestamp
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_file = f"evaluation_results_{timestamp}.xlsx"
    result_df = pd.DataFrame({
        'Hit@k': [f"Hit@{k}" for k in top_k_values],
        'Hit Ratio': [hit_ratios[k] for k in top_k_values],
        'MRR': [mrr_scores[k] for k in top_k_values]
    })
    
    result_df.to_excel(output_file, index=False)
    return hit_ratios, mrr_scores

# Chạy đánh giá
excel_path = excel  # Đường dẫn tới file Excel
print("Bắt đầu đánh giá...")
hit_ratios, mrr_scores = evaluate(excel_path)

# In kết quả
print("\nKết quả đánh giá:")
for k in sorted(hit_ratios.keys()):
    print(f"Hit@{k}: {hit_ratios[k]:.4f}")
    print(f"MRR@{k}: {mrr_scores[k]:.4f}")

Bắt đầu đánh giá...


Processing: Chiến lược phát triển của Đại ...:  20%|██        | 2/10 [00:00<00:02,  3.25it/s, Hit@20=1.00]

correct source: {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}
search_results: [{'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Thông điệp của giám đốc đại học.txt', 'id': '4'}, 'level': 0}, {'source': {'source': 'Chương trình đào tạo kỹ sư chuyên sâu đặc thù của Đại học Bách khoa Hà Nội.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '5'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '1'}, 'level': 0}, {'source': {'source': 'Sứ mạng, tầm nhìn và giá trị cốt lõi.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Các giai đoạn lịch sử.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Quy định miễn giảm học phí 2023.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Trung tâm Công nghệ và Giải pháp Ch

Processing: Mô hình quản trị tiên tiến và ...:  30%|███       | 3/10 [00:00<00:01,  3.87it/s, Hit@20=1.00]

correct source: {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}
search_results: [{'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Thông điệp của giám đốc đại học.txt', 'id': '4'}, 'level': 0}, {'source': {'source': 'Chương trình đào tạo kỹ sư chuyên sâu đặc thù của Đại học Bách khoa Hà Nội.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '5'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '1'}, 'level': 0}, {'source': {'source': 'Hệ thống doanh nghiệp của Đại học.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Mô hình và chương trình đào tạo hệ Đại học chính quy.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Chương trình đào tạo kỹ sư chuyên sâu đặc thù của Đại học Bách khoa Hà Nội.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Các giai đoạn lịch sử.txt', 'id': '0'}, 'level': 0},

...

correct source: {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '2'}
search_results: [{'source': {'source': 'Trung tâm Công nghệ và Giải pháp Chuyển đổi số trong Giáo dục (EdTech).txt', 'id': '2'}, 'level': 0}, {'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '2'}, 'level': 0}, {'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '1'}, 'level': 0}, {'source': {'source': 'Hướng dẫn triển khai các vấn đề liên quan đến CLB Sinh viên.txt', 'id': '0'}, 'level': 0}, {'source': {'source': 'Quy chế đào tạo 2023.txt', 'id': '5'}, 'level': 0}, {'source': {'source': 'Chương trình đào tạo kỹ sư chuyên sâu đặc thù của Đại học Bách khoa Hà Nội.txt', 'id': '1'}, 'level': 0}, {'source': {'source': 'Quy định chuẩn ngoại ngữ đầu ra đối với các chương trình đặc biệt và chất lượng cao.txt', 'id': '0'}, 'level': 0}, {'sourc

Processing: Mô hình quản trị đại học được ...:  90%|█████████ | 9/10 [00:02<00:00,  4.95it/s, Hit@20=1.00]

correct source: {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '2'}
search_results: [{'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '2'}, 'level': 0}, {'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '1'}, 'level': 0}, {'source': {'source': 'Chương trình đào tạo kỹ sư chuyên sâu đặc thù của Đại học Bách khoa Hà Nội.txt', 'id': '13'}, 'level': 0}, {'source': {'source': 'Chiến lược phát triển đại học Bách khoa Hà Nội.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Quy chế đào tạo 2023.txt', 'id': '56'}, 'level': 0}, {'source': {'source': 'Trường Công nghệ Thông tin và Truyền thông.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Hoạt động sinh viên nghiên cứu khoa học Trường Công nghệ Thông tin và Truyền thông.txt', 'id': '3'}, 'level': 0}, {'source': {'source': 'Trường Công nghệ Thông tin và Truyền thông.txt', 'id': '39'}, 'level': 0}, {'source': {'source': 'Quy chế đào tạo 2023.txt', 

Processing: Mô hình quản trị đại học được ...: 100%|██████████| 10/10 [00:02<00:00,  4.44it/s, Hit@20=1.00]


Kết quả đánh giá:
Hit@1: 0.6000
MRR@1: 0.6000
Hit@5: 1.0000
MRR@5: 0.7500
Hit@10: 1.0000
MRR@10: 0.7500
Hit@20: 1.0000
MRR@20: 0.7500
